In [2]:
#Import packages
#---------------------------------------
import sys
import os
import pandas as pd
from matplotlib import pyplot as plt    
import numpy as np
import matplotlib
import warnings
import random
import arviz as az
import pymc as pm
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#Import your modules
#---------------------------------------
import admin_functions as adfn
import cell_decomp_func as cdfn


# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
%load_ext autoreload
sys.version

'3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]'

In [3]:
clust_list = np.linspace(3, 30, 7).astype(int)
genes_list = np.linspace(100, 3000, 7).astype(int)
cells_list = np.linspace(100, 3000, 7).astype(int)
list_list = [clust_list, genes_list, cells_list]
mode_list= ['nclusts', 'ngenes', 'nspots']

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [5]:
# Run alpha noise
import time

model_type = 'epsilon' #CHANGE!

for x,li in enumerate(list_list):
    mode = mode_list[x]

    for l in li:
        #Set default parameters
        n_clusts = 5
        n_genes = 800
        n_cells = 100
        if mode == 'nclusts':
            n_clusts = l
        if mode == 'ngenes':
            n_genes = l
        if mode == 'nspots':
            n_cells=l

        rate_range = 0,40 #max and min of uniform distribution for generating rates
        noise_type = 'exponential' #CHANGE!

        start_time = time.time()
        #Simulate spot data from simulated gene expression
        spot_sim = cdfn.simulate_cell_mix(n_clusts, n_cells, n_genes).simulate_gene_exp(rate_range, 0)
        n_spots = spot_sim.__dict__['n_spots']
        spots = spot_sim.__dict__['spots']
        ref_exp = spot_sim.__dict__['mean_exps']
        prop_vec = spot_sim.__dict__['prop_vec']
        spots = cdfn.add_noise(spots, per=None, a_std=5, g_std=5, e_std=5, noise_type=noise_type) #CHANGE!

        model = cdfn.epsilon_pymc(n_clusts, n_spots, n_genes, ref_exp, spots, noise_type=noise_type) #CHANGE!
        with model: data=pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)

        post, r2 = cdfn.model_stats(data, prop_vec, n_clusts)
        if len(str(l))<2:pref = '0' + str(l)
        else:pref= str(l)
        fin = (time.time() - start_time)
        if mode=='n_spots':
            l = str(n_spots)
        np.save(s_data + 'spatial_transcriptomics/RCTD-test_noise-' + noise_type + '_model-' + model_type + '_' + mode + '-' + pref + '.npy', np.array([prop_vec, post, r2, fin], dtype=object))
        print(l)


05 5
06 6
08 8
10 10
13 13
17 17
23 23
29 29
38 38
50 50
